## [문자]청와대 청원 : 청원의 주제가 무엇일까?

텍스트 데이터를 분석하여 청원 주제를 분류 해보세요.
### 1. train.csv / test.csv
- index: index
- category: 청원 주제/범주
- data: 청원 내용

### 2. submission.csv (제출 파일 형식)

### 3. 라벨 종류
- 0 : 인권/성평등
- 1 : 문화/예술/체육/언론
- 2 : 육아/교육

### NLP(자연어 처리)
1. 한국어는 영어와 달리 단어별로 토큰화를 이루게 되면 해석이 달라지므로 ***형태소 토큰화***가 이루어져야 합니다.
2. ex) 문장: 에디가 책을 읽었다.
 - 단어 토큰화: ['에디가', '책을', '읽었다']
 - 형태소 토큰화: 자립 형태소['에디', '책'], 의존 형태소['-가', '-을', '읽-', '-었', '-다']
3. 전처리 과정에서 제거해야 하는 데이터는 '자연어가 아니면서 아무 의미도 갖지 않는 글자(특수 문자 등)와 분석하고자 하는 목적에 맞지 않는 불필요 단어들입니다.
4. 한국어 전처리 패키지
 - PyKoSpacing: 띄어쓰기가 되어있지 않은 문장을 띄어쓰기를 한 문장으로 변환해주는 패키지입니다.
 - Py-Hanspell: 네이버의 '맞춤법 검사기' 바탕으로 만들어진 패키지로 띄어쓰기와 맞춤법을 지원합니다.
 - SOYNLP: 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저(단어 토큰화 모델)입니다.
 - KoNLPy: 한나눔(Hannanum), 꼬꼬마(Kkma), 메캅(Mecab), 코모란(Komoran), okt(Twitter)와 같은 형태소 분석기를 제공하는 패키지입니다.

#### `1.` 라이브러리 임포트

In [6]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt # 한국어 자연어 처리를 위한 형태소 분석기 입니다.

#### `2.` 데이터 로드

In [2]:
train_data = pd.read_csv('./청와대 청원/train.csv')
test_data = pd.read_csv('./청와대 청원/test.csv')
submission_data = pd.read_csv('./청와대 청원/sample_submission.csv')

#### `3.` 데이터 정보 확인

In [3]:
display(train_data.head())
print('train_data.info()', train_data.info())
display(test_data.head())
print('test_data.info()', test_data.info())
display(submission_data.head())
print('submission_data.info()', submission_data.info())

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     40000 non-null  int64 
 1   category  40000 non-null  int64 
 2   data      39992 non-null  object
dtypes: int64(2), object(1)
memory usage: 937.6+ KB
train_data.info() None


,index,data
0,0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...
1,1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...
2,2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,3,국민위원에가 삼성편만들어요. 삼성에서 11년간 일하고 혈암과 백혈병 진단을 받은 ...
4,4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   5000 non-null   int64 
 1   data    5000 non-null   object
dtypes: int64(1), object(1)
memory usage: 78.2+ KB
test_data.info() None


,index,category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   index     5000 non-null   int64
 1   category  5000 non-null   int64
dtypes: int64(2)
memory usage: 78.2 KB
submission_data.info() None


#### `4-1.` 데이터 전처리

In [4]:
# 데이터의 정보를 확인한 결과 data에 NaN 값이 존재합니다. 데이터의 특성상 결측값을 채우기에는 어려운 부분이 있고
# 결측값의 개수가 적으므로 결측값이 있는 행을 삭제하겠습니다.
train_data = train_data.dropna()

# train_data와 test_data의 값을 확인해 보면 '\\n'을 확인할 수 있습니다. 이는 이스케이프(Escape) 문자로 \(backslash)를
# 앞에 붙여 php에 정의되어 있는 원래의 의미를 벗어나는 문자입니다. 여기서 '\\n'은 줄 바꿈을 의미합니다.
# 이스케이프 문자를 제거하기 위해 한글의 범위를 지정할 수 있는 정규 표현식을 이용하겠습니다.
# 자음의 범위: ㄱ ~ ㅎ, 모음의 범위:ㅏ ~ ㅣ, 완성형 한글의 범위: 가 ~ 힣, ^: not을 의미함
# train_data['data']열의 값들을 string형으로 바꾼 뒤 정규 표현식 범위의 해당되지 않는 것들을 공백으로 바꿉니다.
train_data['data'] = train_data['data'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_data['data'] = test_data['data'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')

# 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39992 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     39992 non-null  int64 
 1   category  39992 non-null  int64 
 2   data      39992 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


#### `4-2.` 데이터 전처리: 형태소 분석기를 이용한 토큰화 및 정규화 작업

In [ ]:
# 데이터의 문장을 하나씩 가져와서 sentence에 넘겨주고

X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

#### 참고 사이트
1. https://wikidocs.net/book/2155
2. https://www.itworld.co.kr/news/187793
3. https://dacon.io/competitions/open/235597/codeshare/955?page=1&dtype=recent